| [07_information_extraction/01_CRF实体识别模型.ipynb](https://github.com/shibing624/nlp-tutorial/tree/main/07_information_extraction/01_CRF实体识别模型.ipynb)  | 从头实现CRF实体识别模型  |[Open In Colab](https://colab.research.google.com/github/shibing624/nlp-tutorial/blob/main/07_information_extraction/01_CRF实体识别模型.ipynb) |

# CRF的中文实体识别模型

非结构化的文本内容有很多丰富的信息，但找到相关的知识始终是一个具有挑战性的任务，命名实体识别可以帮忙找到其中核心的实体词，对新词发现也有挖掘扩展作用。

前面我们用隐马尔可夫模型（HMM）自己尝试训练过一个分词器，其实 HMM 也可以用来训练命名实体识别器，但在本节，我们讲另外一个算法——条件随机场（CRF），来训练一个命名实体识别器。

## 条件随机场（CRF）

条件随机场（Conditional Random Fields，简称 CRF）是给定一组输入序列条件下另一组输出序列的条件概率分布模型，在自然语言处理中得到了广泛应用。

首先，我们来看看什么是随机场。“随机场”的名字取的很玄乎，其实理解起来不难。随机场是由若干个位置组成的整体，当按照某种分布给每一个位置随机赋予一个值之后，其全体就叫做随机场。

对于 CRF，我们给出准确的数学语言描述：设 X 与 Y 是随机变量，P(Y|X) 是给定 X 时 Y 的条件概率分布，若随机变量 Y 构成的是一个马尔科夫随机场，则称条件概率分布 P(Y|X) 是条件随机场。

## 基于 CRF 的中文命名实体识别模型实现

在常规的命名实体识别中，通用场景下最常提取的是时间、人物、地点及组织机构名，因此本模型也将提取以上四种实体。


引入库：

In [1]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import re
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import pickle

In [3]:
class CorpusProcess(object):

    def __init__(self, data_dir='data/'):
        """初始化"""
        self.train_corpus_path = data_dir + "1980_01_rmrb.txt"
        self.process_corpus_path = data_dir + "result_rmrb.txt"
        self._maps = {'t': 'T','nr': 'PER', 'ns': 'ORG','nt': 'LOC'}
        
    def read_corpus_from_file(self, file_path):
        """读取语料"""
        f = open(file_path, 'r',encoding='utf-8')
        lines = f.readlines()
        f.close()
        return lines
    
    def write_corpus_to_file(self, data, file_path):
        """写语料"""
        f = open(file_path, 'wb')
        f.write(data)
        f.close()
        
    def q_to_b(self,q_str):
        """全角转半角"""
        b_str = ""
        for uchar in q_str:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif 65374 >= inside_code >= 65281:  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            b_str += chr(inside_code)
        return b_str
    
    def b_to_q(self,b_str):
        """半角转全角"""
        q_str = ""
        for uchar in b_str:
            inside_code = ord(uchar)
            if inside_code == 32:  # 半角空格直接转化
                inside_code = 12288
            elif 126 >= inside_code >= 32:  # 半角字符（除空格）根据关系转化
                inside_code += 65248
            q_str += chr(inside_code)
        return q_str
    
    def pre_process(self):
        """语料预处理 """
        lines = self.read_corpus_from_file(self.train_corpus_path)
        new_lines = []
        for line in lines:
            words = self.q_to_b(line.strip()).split('  ')
            pro_words = self.process_t(words)
            pro_words = self.process_nr(pro_words)
            pro_words = self.process_k(pro_words)
            new_lines.append('  '.join(pro_words[1:]))
        self.write_corpus_to_file(data='\n'.join(new_lines).encode('utf-8'), file_path=self.process_corpus_path)
    
    def process_k(self, words):
        """处理大粒度分词,合并语料库中括号中的大粒度分词,类似：[国家/n  环保局/n]nt """
        pro_words = []
        index = 0
        temp = ''
        while True:
            word = words[index] if index < len(words) else ''
            if '[' in word:
                temp += re.sub(pattern='/[a-zA-Z]*', repl='', string=word.replace('[', ''))
            elif ']' in word:
                w = word.split(']')
                temp += re.sub(pattern='/[a-zA-Z]*', repl='', string=w[0])
                pro_words.append(temp+'/'+w[1])
                temp = ''
            elif temp:
                temp += re.sub(pattern='/[a-zA-Z]*', repl='', string=word)
            elif word:
                pro_words.append(word)
            else:
                break
            index += 1
        return pro_words
    
    def process_nr(self, words):
        """ 处理姓名，合并语料库分开标注的姓和名，类似：温/nr  家宝/nr"""
        pro_words = []
        index = 0
        while True:
            word = words[index] if index < len(words) else ''
            if '/nr' in word:
                next_index = index + 1
                if next_index < len(words) and '/nr' in words[next_index]:
                    pro_words.append(word.replace('/nr', '') + words[next_index])
                    index = next_index
                else:
                    pro_words.append(word)
            elif word:
                pro_words.append(word)
            else:
                break
            index += 1
        return pro_words
    
    def process_t(self, words):
        """处理时间,合并语料库分开标注的时间词，类似： （/w  一九九七年/t  十二月/t  三十一日/t  ）/w   """
        pro_words = []
        index = 0
        temp = ''
        while True:
            word = words[index] if index < len(words) else ''
            if '/t' in word:
                temp = temp.replace('/t', '') + word
            elif temp:
                pro_words.append(temp)
                pro_words.append(word)
                temp = ''
            elif word:
                pro_words.append(word)
            else:
                break
            index += 1
        return pro_words
    
    def pos_to_tag(self, p):
        """由词性提取标签"""
        t = self._maps.get(p, None)
        return t if t else 'O'
    
    def tag_perform(self, tag, index):
        """标签使用BIO模式"""
        if index == 0 and tag != 'O':
            return 'B_{}'.format(tag)
        elif tag != 'O':
            return 'I_{}'.format(tag)
        else:
            return tag
        
    def pos_perform(self, pos):
        """去除词性携带的标签先验知识"""
        if pos in self._maps.keys() and pos != 't':
            return 'n'
        else:
            return pos
        
    def initialize(self):
        """初始化 """
        lines = self.read_corpus_from_file(self.process_corpus_path)
        words_list = [line.strip().split('  ') for line in lines if line.strip()]
        del lines
        self.init_sequence(words_list)
        
    def init_sequence(self, words_list):
        """初始化字序列、词性序列、标记序列 """
        words_seq = [[word.split('/')[0] for word in words] for words in words_list]
        pos_seq = [[word.split('/')[1] for word in words] for words in words_list]
        tag_seq = [[self.pos_to_tag(p) for p in pos] for pos in pos_seq]
        self.pos_seq = [[[pos_seq[index][i] for _ in range(len(words_seq[index][i]))]
                        for i in range(len(pos_seq[index]))] for index in range(len(pos_seq))]
        self.tag_seq = [[[self.tag_perform(tag_seq[index][i], w) for w in range(len(words_seq[index][i]))]
                        for i in range(len(tag_seq[index]))] for index in range(len(tag_seq))]
        self.pos_seq = [['un']+[self.pos_perform(p) for pos in pos_seq for p in pos]+['un'] for pos_seq in self.pos_seq]
        self.tag_seq = [[t for tag in tag_seq for t in tag] for tag_seq in self.tag_seq]
        self.word_seq = [['<BOS>']+[w for word in word_seq for w in word]+['<EOS>'] for word_seq in words_seq]   
        
    def extract_feature(self, word_grams):
        """特征选取"""
        features, feature_list = [], []
        for index in range(len(word_grams)):
            for i in range(len(word_grams[index])):
                word_gram = word_grams[index][i]
                feature = {'w-1': word_gram[0], 'w': word_gram[1], 'w+1': word_gram[2],
                           'w-1:w': word_gram[0]+word_gram[1], 'w:w+1': word_gram[1]+word_gram[2],
                           # 'p-1': self.pos_seq[index][i], 'p': self.pos_seq[index][i+1],
                           # 'p+1': self.pos_seq[index][i+2],
                           # 'p-1:p': self.pos_seq[index][i]+self.pos_seq[index][i+1],
                           # 'p:p+1': self.pos_seq[index][i+1]+self.pos_seq[index][i+2],
                           'bias': 1.0}
                feature_list.append(feature)
            features.append(feature_list)
            feature_list = []
        return features 
    
    def segment_by_window(self, words_list=None, window=3):
        """窗口切分"""
        words = []
        begin, end = 0, window
        for _ in range(1, len(words_list)):
            if end > len(words_list): break
            words.append(words_list[begin:end])
            begin = begin + 1
            end = end + 1
        return words
    
    def generator(self):
        """训练数据"""
        word_grams = [self.segment_by_window(word_list) for word_list in self.word_seq]
        features = self.extract_feature(word_grams)
        return features, self.tag_seq

In [4]:
class CRF_NER(object):

    def __init__(self, data_dir='data/'):
        """初始化参数"""
        self.algorithm = "lbfgs"
        self.c1 ="0.1"
        self.c2 = "0.1"
        self.max_iterations = 100
        self.model_path = data_dir + "model.pkl"
        self.corpus = CorpusProcess(data_dir)  #Corpus 实例
        self.corpus.pre_process()  #语料预处理
        self.corpus.initialize()  #初始化语料
        self.model = None

    def initialize_model(self):
        """初始化"""
        algorithm = self.algorithm
        c1 = float(self.c1)
        c2 = float(self.c2)
        max_iterations = int(self.max_iterations)
        self.model = sklearn_crfsuite.CRF(algorithm=algorithm, c1=c1, c2=c2,
                                          max_iterations=max_iterations, all_possible_transitions=True)

    def train(self):
        """训练"""
        self.initialize_model()
        x, y = self.corpus.generator()
        x_train, y_train = x[500:], y[500:]
        x_test, y_test = x[:500], y[:500]
        self.model.fit(x_train, y_train)
        labels = list(self.model.classes_)
        labels.remove('O')
        y_predict = self.model.predict(x_test)
        metrics.flat_f1_score(y_test, y_predict, average='weighted', labels=labels)
        sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
        print(metrics.flat_classification_report(y_test, y_predict, labels=sorted_labels, digits=3))
        self.save_model()

    def predict(self, sentence):
        """预测"""
        self.load_model()
        u_sent = self.corpus.q_to_b(sentence)
        word_lists = [['<BOS>']+[c for c in u_sent]+['<EOS>']]
        word_grams = [self.corpus.segment_by_window(word_list) for word_list in word_lists]
        features = self.corpus.extract_feature(word_grams)
        y_predict = self.model.predict(features)
        entity = ''
        for index in range(len(y_predict[0])):
            if y_predict[0][index] != 'O':
                if index > 0 and y_predict[0][index][-1] != y_predict[0][index-1][-1]:
                    entity += ' '
                entity += u_sent[index]
            elif entity[-1] != ' ':
                entity += ' '
        return entity

    def load_model(self):
        """加载模型 """
        self.model = pickle.load(open(self.model_path, 'rb'))

    def save_model(self):
        """保存模型"""
        pickle.dump(self.model, open(self.model_path, 'wb'))

In [5]:
ner = CRF_NER(data_dir='data/')
ner

In [6]:
model = ner.train()

/Users/xuming/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B_LOC', 'I_LOC', 'B_ORG', 'I_ORG', 'B_PER', 'I_PER', 'B_T', 'I_T'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


              precision    recall  f1-score   support

       B_LOC      0.929     0.492     0.644       266
       I_LOC      0.807     0.420     0.552      1203
       B_ORG      0.861     0.670     0.754       682
       I_ORG      0.844     0.603     0.703       997
       B_PER      0.929     0.330     0.487       440
       I_PER      0.949     0.364     0.526       824
         B_T      0.925     0.831     0.875       444
         I_T      0.953     0.902     0.927      1099

   micro avg      0.892     0.588     0.709      5955
   macro avg      0.900     0.576     0.683      5955
weighted avg      0.889     0.588     0.689      5955



In [7]:
ner.predict('新华社北京十二月三十一日电(中央人民广播电台记者刘振英、新华社记者张宿堂)今天是一九九七年的最后一天。')

'新华社 北京 十二月三十一日  中央人民广播电台  刘振英  新华社  张宿堂  今天  一九九七年 '

In [8]:
ner.predict(u'一九四九年，国庆节，毛泽东同志在天安门城楼上宣布中国共产党从此站起来了！')

'一九四九年  毛泽东  中国 '

本节完。